In [3]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
        
def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    #img = cv2.cvtColor(img , cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(img , cv2.CV_64F , 1,0 , ksize=sobel_kernel )
    sobley = cv2.Sobel(img , cv2.CV_64F , 0 ,1 ,  ksize=sobel_kernel)
    abs_sobelx = np.sqrt( sobelx**2 + sobley**2)
    scale_factor = np.max(abs_sobelx)/255 
    scaled_sobel = (abs_sobelx/scale_factor).astype(np.uint8) 
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    return binary_output
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img1 = cv2.imread(fname)
    img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(img, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1], None, None)
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        # hls = cv2.cvtColor(dst , cv2.COLOR_RGB2HLS)
        binary_output = mag_thresh(dst, sobel_kernel=3, thresh=(0, 255))
        #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        #ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
        cv2.imshow('imgbinary',binary_output)
        
        cv2.waitKey(500)
        cv2.imshow('img',dst)
        cv2.waitKey(500)

    
cv2.destroyAllWindows()